In [2]:
#%pip install beautifulsoup4
#%pip install pip-system-certs
#% -m pip install ipykernel -U --force-reinstall

In [10]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import os
from langchain_community.graphs import Neo4jGraph
from PyPDF2 import PdfReader
import fitz
from langdetect import detect
import spacy
import re
from unidecode import unidecode
import time

Conectando ao Neo4j 

In [11]:
from neo4j import GraphDatabase

# Neo4j variables
NEO4J_URL = os.getenv("NEO4J_URL")
NEO4J_USERNAME =os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")


graph = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

- Buscar os links para a tese no Neo4j
- Buscar o link do PDF no repositório da universidade
- Baixar o PDF
- Extrair o texto do PDF 

Buscar links para a tese no Neo4j

In [12]:
# Query to get theses and their repositories from the Neo4j database
# This query retrieves theses that are published by the Federal University of São Carlos (UFSCar) and have not been processed yet.
query1 = """
MATCH (t:Thesis)-[:publisher]-(uni:University{uri:'tag:stardog:api:UFSCAR'})
WHERE NOT EXISTS {(t{processed:true})}
RETURN distinct(t["uri"]) as Thesis, t["repository"] as repository 
"""

# Alternative query to get theses that do not have excerpts and have been processed.
query2 = """ 
match (t:Thesis)-[:publisher]-(uni:University{uri:'tag:stardog:api:UFSCAR'})
where (not (t:Thesis)-[:BFO_0000050]-(:Exerpt)) and  exists {(t{processed:true})}
return distinct(t["uri"]) as Thesis, t["repository"] as repository
"""

# Query to retrive theses with few pages and their repositories.
query3 = """
MATCH (t:Thesis)-[:publisher]-(uni:University{uri:'tag:stardog:api:UFSCAR'})
Match (t:Thesis)-[:BFO_0000050]-(e:Exerpt)
RETURN count(e) as pages, t["uri"] as Thesis, t["repository"] as repository
 order by (pages) limit 40
"""

query4 = """
MATCH (t:Thesis)-[:publisher]-(uni:University{uri:'tag:stardog:api:UFSCAR'})
WHERE NOT EXISTS {(t{processed:true})}
RETURN distinct(t["uri"]) as Thesis, t["title"] as title 
"""

query5 = """
MATCH (t:Thesis {processed: true})-[:publisher]-(uni:University {uri: 'tag:stardog:api:UFSCAR'})
WHERE NOT (t)--(:Exerpt)
RETURN distinct(t["uri"]) as Thesis, t["title"] as title 
"""

graph = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
results = graph.execute_query(query1).records
graph.close()

#results = graph.query(query2)
# Convert the results to a DataFrame
#thesis_repository = pd.DataFrame(results, columns=["pages", "Thesis", "repository"])
thesis_repository = pd.DataFrame(results, columns=["Thesis", "repository"])
thesis_repository = thesis_repository.reset_index(drop=True) 
thesis_repository

,Thesis,repository
0,tag:stardog:api:transformacoes_politicas_e_ind...,https://repositorio.ufscar.br/handle/20.500.14...
1,tag:stardog:api:sao_benedito_e_a_estacao_das_c...,https://repositorio.ufscar.br/handle/20.500.14...
2,tag:stardog:api:direitos_humanos_e_conservador...,https://repositorio.ufscar.br/handle/20.500.14...
3,tag:stardog:api:coletivo_de_estudos_e_acoes_in...,https://repositorio.ufscar.br/handle/20.500.14...
4,tag:stardog:api:reflexoes_e_possibilidades_sob...,https://repositorio.ufscar.br/handle/20.500.14...
...,...,...
360,tag:stardog:api:africanidades_e_educacao_popul...,https://repositorio.ufscar.br/handle/20.500.14...
361,tag:stardog:api:relatos_trajetorias_e_imagens_...,https://repositorio.ufscar.br/handle/20.500.14...
362,tag:stardog:api:de_coisa_a_sujeito__o_processo...,https://repositorio.ufscar.br/handle/20.500.14...
363,tag:stardog:api:o_movimento_negro_e_a_cultura_...,https://repositorio.ufscar.br/handle/20.500.14...


In [13]:
repo = thesis_repository['repository'][0]
print(repo)
get_pdf_link(repo)

https://repositorio.ufscar.br/handle/20.500.14289/7663


['https://repositorio.ufscar.br/server/api/core/bitstreams/a6bc209c-44dc-4f5a-bc96-e0b36752da97/content']

Criando as utils functions

In [14]:
# Funcao para coletar link do repositorio na página da BDTD

#def get_repository_link(title):

#    bdtd_url = 'https://bdtd.ibict.br/vufind/Search/Results?lookfor=' + title.replace(' ', '+') + '&type=Title' #&limit=1'
    #Fazer requisição e parsear o arquivo html
#    try:
#        f = requests.get(bdtd_url, verify=True).text#, verify=False) 
#    except:
#        print("Sleeping for 5 seconds before retrying... (get_repository_link)")
#        time.sleep(5)
#        try:
#            f = requests.get(bdtd_url, verify=True).text#, verify=False) 
#        except:
#            print("Sleeping for 30 seconds before retrying... (get_repository_link)")
#            time.sleep(30)
#            f = requests.get(bdtd_url, verify=True).text#, verify=False)    
#    soup = bs(f, "html.parser")

    #Coletando link para o repositório
#    link_list = []
#    for doc in soup.find_all('a', href=True):

#        if (doc['href'].startswith('https://repositorio.ufsc.br/') |
#            doc['href'].startswith('http://repositorio.ufsc.br/')):
#            path = doc['href']
#            link_list.append(path)
#    if link_list != []:
#        return list(set(link_list))[0]
#    else:
#        return None



# Funcao para coletar link do pdf

def get_pdf_link(url):
    
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
                }

    #Fazer requisição e parsear o arquivo html
    try:
        f = requests.get(url, headers=headers, verify=True).text#, verify=False) 
        
    except:
        print("Sleeping for 5 seconds before retrying... (get_pdf_link)")
        time.sleep(5)
        try:
            f = requests.get(url, headers=headers, verify=True).text#, verify=False) 
        except:
            print("Sleeping for 30 seconds before retrying... (get_pdf_link)")
            time.sleep(30)
            f = requests.get(url, headers=headers, verify=True).text#, verify=False)    
    soup = bs(f, "html.parser")
    
    #Coletando link para as teses
    link_list = []
    for doc in soup.find_all('a', href=True):
        if doc['href'].startswith('/bitstreams/'):     # (doc['href'].endswith('.pdf')):# & doc['href'].startswith('/bitstream')):
            path = doc['href']
            prefix_uni = 'https://repositorio.ufscar.br/server/api/core'
            link = prefix_uni + path.replace('download', 'content')
            link_list.append(link)
    if link_list != []:
        return list(set(link_list))
    else:
        return None
    
    #return None

#fazer download do arquivo
def download_file(pdf_link, thesis_uri):

    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
                }
    
    # NOTE the stream=True parameter below
    try:
        r = requests.get(pdf_link, headers=headers, verify=True, stream=True)
    except:
        print("Sleeping for 5 seconds before retrying... (download_file)")
        time.sleep(5)
        try:
            r = requests.get(pdf_link, headers=headers, verify=True, stream=True)
        except:
            print("Sleeping for 30 seconds before retrying... (download_file)")
            time.sleep(30)
            r = requests.get(pdf_link, headers=headers, verify=True, stream=True)
    # raise_for_status() is not needed with stream=True
    # r.raise_for_status()
    with open('tese UFSCAR.pdf', 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)
    return 

#Carregando os modelos SpaCy para inglês e português 
nlp_en = spacy.load("en_core_web_lg")
nlp_pt = spacy.load("pt_core_news_lg")

# Funcão que recebe uma string e a limpar para ficar no formato aceitável para uma URI
def process_uri(x):
    return (re.sub('[^a-zA-Z0-9_ ]', '',
            unidecode(x.strip())
                        .replace(" ", "_")
                        .replace("[","")
                        .replace("]","")
                        .replace("?","")
                        .replace("'","")
                        .lower()))


# Função para extrair texto de uma tese em PDF e identificar entidades
def extratc_text(tese_path):
    
    # Processando o arquivo em PDF
    try:
        reader = PdfReader(tese_path)
        number_of_pages = len(reader.pages)
    
        excerpt = {}

        extractor = 'PdfReader'

    except:
        reader = fitz.open(tese_path)
        number_of_pages = len(reader)
    
        excerpt = {}
        
        extractor = 'PyMuPDF'

    # Iterando por cada página
    for page_number in range(number_of_pages):

        # Verificando qual extrator de pdf foi utilizado    
        if extractor == 'PdfReader':

            # Extraindo o texto da página
            try:
                if page_number == 0:
                    try:
                        page_text = (reader.pages[page_number].extract_text() + '\n ' 
                                    + reader.pages[page_number+1].extract_text()[:400])
                    except:
                        page_text = reader.pages[page_number].extract_text()
                else:
                    
                    if page_number == number_of_pages -1:
                        try:
                            page_text = (reader.pages[page_number-1].extract_text()[-400:] + '\n ' 
                                        + reader.pages[page_number].extract_text())
                        except:
                            page_text = reader.pages[page_number].extract_text()
                    
                    else:
                        try:
                            page_text = (reader.pages[page_number-1].extract_text()[-400:] + '\n ' 
                                        + reader.pages[page_number].extract_text() + '\n ' 
                                        + reader.pages[page_number+1].extract_text()[:400])  
                        except:
                            page_text = reader.pages[page_number].extract_text()
            except:
                page_text = ''
            
        else:
                
            # Extraindo o texto da página
            try:
                if page_number == 0:
                    try:
                        page_text = (reader[page_number].get_text() + '\n ' 
                                    + reader[page_number+1].get_text()[:400])
                    except:
                        page_text = reader[page_number].get_text()
                else:
                    
                    if page_number == number_of_pages -1:
                        try:
                            page_text = (reader[page_number-1].get_text()[-400:] + '\n ' 
                                        + reader[page_number].get_text())
                        except:
                            page_text = reader[page_number].get_text()
                    
                    else:
                        try:
                            page_text = (reader[page_number-1].get_text()[-400:] + '\n ' 
                                        + reader[page_number].get_text() + '\n ' 
                                        + reader[page_number+1].get_text()[:400])  
                        except:
                            page_text = reader[page_number].get_text()

            except:
                page_text = ''

        # Limpando page text         
        page_text = page_text.replace("'", "").replace('"', '')
        # Verifica se o último caractere é uma barra invertida
        if page_text != '':
            if page_text[-1] == '\\': 
                page_text = page_text[:-2]
        page_text = re.sub(r"[\ud800-\udfff]", " ", page_text)
        page_text = page_text.encode('utf-8').decode('utf-8')
        
        
        # Detectando o idioma do texto
        try:
            lang = detect(page_text)
            
        except:
            lang = 'Não_detectado'
        
        
        # Extraindo as entidades
        page_extracted = False
        if lang == 'pt':
            doc = nlp_pt(page_text)
            page_extracted = True
        if lang == 'en':
            doc = nlp_en(page_text)
            page_extracted = True
        

        if page_extracted:

            persons = []
            gpes = []
            #orgs = []
            for ent in doc.ents:
                if ent.label_ == "PER": #"PERSON":
                    persons.append(process_uri(ent.text))
                if ent.label_ == "LOC": #"GPE":
                    gpes.append(process_uri(ent.text))
                #if ent.label_ == "ORG":
                #    orgs.append(process_uri(ent.text))
                    
            excerpt[page_number] = {'text': page_text, 
                                    'lang': lang, 
                                    'persons': list(set(persons)), 
                                    'gpes': list(set(gpes)),
                                #'orgs': list(set(orgs))
                                }

    return excerpt

Iterando por todas as URI para extrair o texto e gravar no grafo NEO4j

In [ ]:
# Gravar hora de início do processamento
start_time = pd.Timestamp.now()
parcial_time = pd.Timestamp.now()

for i in range(0,len(thesis_repository)):

    #title ='The problem of identity as it relates to the american black\'s vision of Africa'
    #title = thesis_repository['title'][i].replace("?","").replace('-', ' ').replace('&', ' ').replace('/', ' ').replace('\\', ' ')  
    #repo = get_repository_link(title)

    # Coletando o link do PDF
    url = thesis_repository['repository'][i]
    thesis_uri = thesis_repository['Thesis'][i][16:]
    pdf_link = get_pdf_link(url)
    if pdf_link is not None:
        # Itereando pelos links de PDF encontrados (em alguns casos há outros materiais além da tese)
        for pdf_num in range(len(pdf_link)):
            print(f"{i} - Downloading and extracting: {thesis_uri} from {pdf_link[pdf_num]}")
            download_file(pdf_link[pdf_num], thesis_uri)

            # Extraindo o texto e entidades
            tese_path = 'tese UFSCAR.pdf'
            excerpt = extratc_text(tese_path)

            excerpts = []
            persons = []
            gpes = []

            # Salvando as triplas em CSV
            for n in excerpt.keys():

                excerpt_uri = thesis_repository['Thesis'][i] + '_pdf_' + str(pdf_num) + '_p_' + str(n)
                excerpt_text = excerpt[n]['text']
                # Verifica se o último caractere é uma barra invertida
                #if excerpt_text[-1] == '\\': 
                #    excerpt_text = excerpt_text[:-1]
                

                excerpt_page = 'pdf_' + str(pdf_num) + '_p_' + str(n)
                excerpt_lang = excerpt[n]['lang']
                
                excerpts.append([thesis_repository['Thesis'][i], excerpt_uri, excerpt_text, excerpt_page, excerpt_lang])
                    
                #Criando as relações com as entidades
                for person in excerpt[n]['persons']:
                    persons.append([excerpt_uri, 'tag:stardog:api:' + person])
                
                for gpe in excerpt[n]['gpes']:
                    gpes.append([excerpt_uri, 'tag:stardog:api:' + gpe])
            
            # Salvando os dados em CSV no diretório de importação do Neo4j
            pd.DataFrame(excerpts).to_csv('C:\\Users\\facordei\\.Neo4jDesktop\\relate-data\\dbmss\\dbms-792eabfc-5517-4220-ab29-30ef6ec3236e\\import\\excerptsUFSCAR.csv', index=False, header=False)
            pd.DataFrame(persons).to_csv('C:\\Users\\facordei\\.Neo4jDesktop\\relate-data\\dbmss\\dbms-792eabfc-5517-4220-ab29-30ef6ec3236e\\import\\personsUFSCAR.csv', index=False, header=False)
            pd.DataFrame(gpes).to_csv('C:\\Users\\facordei\\.Neo4jDesktop\\relate-data\\dbmss\\dbms-792eabfc-5517-4220-ab29-30ef6ec3236e\\import\\gpesUFSCAR.csv', index=False, header=False)

            #Deletando o arquivo PDF após o processamento
            os.remove('tese UFSCAR.pdf')

            print(f"Loading triplas: {thesis_uri}")

            # Importanto os dados para o Neo4j
            #Excerpts
            query = """
            LOAD CSV FROM 'file:///excerptsUFSCAR.csv'
            AS row
            MATCH  (t:Thesis{uri: row[0]})
            MERGE  (e:Exerpt{
                    uri: row[1],
                    text: row[2],
                    page: row[3],
                    lang: row[4]
                    })-[r:BFO_0000050]-(t);
            """
            graph = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
            results = graph.execute_query(query)
            graph.close()

            #Persons
            query = """
            LOAD CSV FROM 'file:///personsUFSCAR.csv'
            AS row
            MATCH  (e:Exerpt{uri: row[0]})
            MERGE  (p:Person{uri: row[1]})
            MERGE  (e)-[r:mentions]-(p);
            """
            graph = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
            results = graph.execute_query(query)
            graph.close()

            #GPEs
            query = """
            LOAD CSV FROM 'file:///gpesUFSCAR.csv'
            AS row
            MATCH  (e:Exerpt{uri: row[0]})
            MERGE  (p:Place{uri: row[1]})
            MERGE  (e)-[r:mentions]-(p);
            """
            graph = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
            results = graph.execute_query(query)
            graph.close()

            # Removendo os arquivos CSV após a importação
            os.remove('C:\\Users\\facordei\\.Neo4jDesktop\\relate-data\\dbmss\\dbms-792eabfc-5517-4220-ab29-30ef6ec3236e\\import\\excerptsUFSCAR.csv')
            os.remove('C:\\Users\\facordei\\.Neo4jDesktop\\relate-data\\dbmss\\dbms-792eabfc-5517-4220-ab29-30ef6ec3236e\\import\\personsUFSCAR.csv')
            os.remove('C:\\Users\\facordei\\.Neo4jDesktop\\relate-data\\dbmss\\dbms-792eabfc-5517-4220-ab29-30ef6ec3236e\\import\\gpesUFSCAR.csv')

            print(f"Total time: {pd.Timestamp.now() - start_time}")
            print(f"Partial time: {pd.Timestamp.now() - parcial_time}")
            # Atualizando a hora parcial
            parcial_time = pd.Timestamp.now()
    else:
        print(f"No PDF link found for {thesis_uri}")

    # gravando no grafo que a tese foi processada
    query = """
    MATCH  (t:Thesis{uri: \'""" + thesis_repository['Thesis'][i] +  """\'})
    SET t.processed = true
    """
    graph = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
    results = graph.execute_query(query)
    graph.close()
    #break


0 - Downloading and extracting: transformacoes_politicas_e_indigenas__movimento_e_prefeitura_no_alto_rio_negro from https://repositorio.ufscar.br/server/api/core/bitstreams/a6bc209c-44dc-4f5a-bc96-e0b36752da97/content
Loading triplas: transformacoes_politicas_e_indigenas__movimento_e_prefeitura_no_alto_rio_negro
Total time: 0 days 00:00:35.525554
Partial time: 0 days 00:00:35.525554
1 - Downloading and extracting: sao_benedito_e_a_estacao_das_curas_saude_devocao_e_constituicoes_do_territorio_sulmineiro from https://repositorio.ufscar.br/server/api/core/bitstreams/70e08680-0a36-4e52-bff9-5d4f0e761bfe/content
Loading triplas: sao_benedito_e_a_estacao_das_curas_saude_devocao_e_constituicoes_do_territorio_sulmineiro
Total time: 0 days 00:02:54.650399
Partial time: 0 days 00:02:19.124845
2 - Downloading and extracting: direitos_humanos_e_conservadorismo_contemporaneo_a_agenda_dos_deputados_do_partido_social_liberal_psl_56a_legislatura_20192023 from https://repositorio.ufscar.br/server/api/c

Multiple definitions in dictionary at byte 0x523e0 for key /Info
Multiple definitions in dictionary at byte 0x523ec for key /Info
Multiple definitions in dictionary at byte 0x523f8 for key /Info


Loading triplas: mulheres_negras_e_seus_universos_comunidade_de_vila_bela_terra_conflitos_e_memorias_nos_descaminhos_da_identidade_etnica_19602018
Total time: 0 days 02:35:02.281742
Partial time: 0 days 00:00:05.947760
200 - Downloading and extracting: mulheres_negras_e_seus_universos_comunidade_de_vila_bela_terra_conflitos_e_memorias_nos_descaminhos_da_identidade_etnica_19602018 from https://repositorio.ufscar.br/server/api/core/bitstreams/67238672-5979-4d32-bffa-1ad5175d227d/content
Loading triplas: mulheres_negras_e_seus_universos_comunidade_de_vila_bela_terra_conflitos_e_memorias_nos_descaminhos_da_identidade_etnica_19602018
Total time: 0 days 02:35:58.106029
Partial time: 0 days 00:00:55.824287
201 - Downloading and extracting: turismo_na_amazonia_ocidental_as_transformacoes_do_ecoturismo_na_comunidade_quilombola_de_pedras_negras_vale_do_guapore_rondonia from https://repositorio.ufscar.br/server/api/core/bitstreams/b714fd54-b881-4530-9c75-2bbdf46b4daa/content
Loading triplas: turi